In [ ]:
! pip install requests pandas gspread

In [56]:
from datetime import date, timedelta
import numpy as np
import pandas as pd
from tapdb import TapDBClient
from gsheets import DateSheet
from config import TAPDB_USER, TAPDB_PASS

target_date = (date.today() - timedelta(days=1)).isoformat()
target_date = '2024-06-28'

tapdb = TapDBClient(TAPDB_USER, TAPDB_PASS, target_date=target_date)

target_date = tapdb.target_date.isoformat()
sheet = DateSheet('[RP_EN] Revenue Report', date=target_date)

In [57]:
# active devices
sheet.change_sheet('Daily Re.')
data = tapdb.get_data('RP CN (PRO)', 'active', dim='os', device=True)

sheet.update_cells('活躍設備\n(tap db)', int(data['DAU_DEVICE'].sum()))
sheet.update_cells('活躍設備\n(android)', int(data.loc['Android', 'DAU_DEVICE']))
data

,key,DAU_DEVICE,newDAU_DEVICE,DAU_DEVICE2,DAU_DEVICE3,DAU_DEVICE7,DAU_DEVICE14,DAU_DEVICE30
key,,,,,,,,
Android,Android,37007,4472,32535,31025,27533,23763,18759
iOS,iOS,4866,3092,1774,762,619,579,415


In [58]:
# new devices
sheet.change_sheet('Daily Re.')
data = tapdb.get_data('RP CN (PRO)', 'source', dim='os', device=True)
sheet.update_cells('新增設備\nTotal\n(Tap DB)', int(data['newDevices'].sum()))
sheet.update_cells('AND新設備\n(Tap DB)', int(data.loc['Android', 'newDevices']))
data


,key,newPlayers,totalCharge,totalChargeAmount,firstChargePlayers,firstChargeAmount,newDevices,convertedDevices,devicesConvertRate,dayRetainedRate1,dayRetainedRate7,chargeRate,firstChargeRate,ARPU,ARPPU
key,,,,,,,,,,,,,,,
Android,Android,5340,100,1576.9,54,439.2,4472,3929,87.86,33.74,None,2.24,1.21,0.10,8.13
iOS,iOS,971,22,267.7,9,20.4,3092,2835,91.69,50.79,None,0.71,0.29,0.01,2.27


In [ ]:
# convert rate by version
data = tapdb.get_data('RP CN (PRO)', 'source', dim='ver')
row = data.loc[max(data.index)]

sheet.change_sheet('轉化率')
sheet.update_cells('版本', row['key'])
sheet.update_cells('新增設備', row['newDevices'])
sheet.update_cells('轉化設備', row['convertedDevices'])

data

In [ ]:
# retain
data = tapdb.get_data('RP CN (PRO)', 'retain', begin_date='2024-04-01', pf='and', type='both')
data.sort_index(inplace=True)
data['key'] = data['key'].astype(str)
data_drr = data.loc[:, data.columns.str.contains('DRR')]
data = pd.concat([data.loc[:, ['newDevices']], data_drr / 100], axis=1)

data.to_csv('~/Desktop/retain.csv')
data

,newDevices,DRR1,DRR2,DRR3,DRR4,DRR5,DRR6,DRR7,DRR8,DRR9,...,DRR26,DRR27,DRR28,DRR29,DRR30,DRR60,DRR90,DRR120,DRR150,DRR180
key,,,,,,,,,,,,,,,,,,,,,
2024-04-01,3392,34.20,25.03,21.14,18.16,16.89,15.18,14.56,13.44,12.03,...,6.04,5.96,5.66,5.22,5.45,2.39,None,None,None,None
2024-04-02,3796,33.11,23.23,19.55,17.52,16.10,14.54,13.94,12.62,11.17,...,5.37,5.22,5.48,4.93,4.69,2.24,None,None,None,None
2024-04-03,6546,32.62,19.75,15.73,12.57,11.21,10.59,9.30,8.66,8.80,...,4.09,5.30,5.27,4.66,3.93,1.82,None,None,None,None
2024-04-04,10555,29.28,17.51,11.22,10.22,9.68,8.53,7.98,7.94,9.50,...,5.34,6.26,4.74,4.18,3.94,1.24,None,None,None,None
2024-04-05,8652,28.51,14.72,12.91,11.53,10.66,9.93,10.53,11.92,9.87,...,6.24,5.61,5.20,4.74,4.52,1.70,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-18,4007,36.21,24.48,21.24,18.22,16.50,14.97,9.58,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None
2024-06-19,3911,36.38,26.23,22.48,18.82,17.39,11.63,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None
2024-06-20,3888,35.83,23.95,20.32,17.46,10.93,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None


In [ ]:
# [RP_EN] Revenue Report >> TapDB >> Android
sheet.change_sheet('TapDB', headers_row=2)
data = tapdb.get_data('RP NTK (PRO)', 'pay', dim='os', device=True,
                flts=[{'k': "dev", 'l': "!=", 'v': ["n2i mono arm arm64 n2i", "n2i multi arm arm64 n2i"]}])

col_to_col = [
    ('收入', 'income', int),
    ('付費人數','paidPlayers', int),
    ('付費次數','payTimes', int),
    ('ARPU','ARPU', float),
    ('ARPPU','ARPPU', float),
    ('付費率','payRate', lambda x: float(x / 100))]
for col_sheet, col_data, func in col_to_col:
    sheet.update_cells(col_sheet, func(data[col_data].sum()))

data

,key,income,paidPlayers,payTimes,refund,refundUsers,refundTimes,ARPU,ARPPU,payRate
key,,,,,,,,,,
Android,Android,16075,579,1422,0,0,0,0.97,27.76,3.49


In [ ]:
# [RP_EN] Revenue Report >> TapDB >> Android
sheet.change_sheet('TapDB', headers_row=2)
data = tapdb.get_data('RP NTK (PRO)', 'active', dim='os', device=True,
                      flts=[{'k': "dev", 'l': "!=", 'v': ["n2i mono arm arm64 n2i", "n2i multi arm arm64 n2i"]}])

sheet.update_cells('活躍裝置', int(data['DAU_DEVICE'].sum()))
sheet.update_cells('新增裝置', int(data['newDAU_DEVICE'].sum()))
data

/var/folders/ct/v7_skn0d1qj1fx21m526kmwm0000gn/T/ipykernel_23746/1679021959.py:7: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  sheet.update_cells('新增裝置', int(data['newDAU_DEVICE']))


,key,DAU_DEVICE,newDAU_DEVICE,DAU_DEVICE2,DAU_DEVICE3,DAU_DEVICE7,DAU_DEVICE14,DAU_DEVICE30
key,,,,,,,,
Android,Android,17634,3199,14435,13622,12044,10326,7866


In [ ]:
# [RP_EN] Revenue Report >> TapDB >> n2i
sheet.change_sheet('TapDB', headers_row=2)
data = tapdb.get_data('RP NTK (PRO)', 'pay', dim='os', device=True,
                      flts=[{'k': "dev", 'l': "=", 'v': ["n2i mono arm arm64 n2i", "n2i multi arm arm64 n2i"]}])

col_to_col = [
    ('收入n2i', 'income', int),
    ('付費人數n2i','paidPlayers', int),
    ('付費次數n2i','payTimes', int),
    ('ARPUn2i','ARPU', float),
    ('ARPPUn2i','ARPPU', float),
    ('付費率n2i','payRate', lambda x: float(x / 100))]
for col_sheet, col_data, func in col_to_col:
    sheet.update_cells(col_sheet, func(data[col_data].sum()))

data

,key,income,paidPlayers,payTimes,refund,refundUsers,refundTimes,ARPU,ARPPU,payRate
key,,,,,,,,,,
Android,Android,3005,90,228,0,0,0,1.57,33.39,4.71


In [60]:
# [RP_EN] Revenue Report >> TapDB >> n2i
sheet.change_sheet('TapDB', headers_row=2)
data = tapdb.get_data('RP NTK (PRO)', 'active', dim='os', device=True,
                      flts=[{'k': "dev", 'l': "=", 'v': ["n2i mono arm arm64 n2i", "n2i multi arm arm64 n2i"]}])

sheet.update_cells('活躍裝置n2i', int(data['DAU_DEVICE'].sum()))
sheet.update_cells('新增裝置n2i', int(data['newDAU_DEVICE'].sum()))
data

,key,DAU_DEVICE,newDAU_DEVICE,DAU_DEVICE2,DAU_DEVICE3,DAU_DEVICE7,DAU_DEVICE14,DAU_DEVICE30
key,,,,,,,,
Android,Android,4531,4525,6,0,0,0,0
